In [4]:
!nvidia-smi

Fri Dec  8 16:21:12 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.90                 Driver Version: 384.90                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    Off  | 00000000:04:00.0 Off |                  N/A |
| 27%   29C    P8     9W / 180W |    821MiB /  8114MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 1080    Off  | 00000000:05:00.0 Off |                  N/A |
| 35%   52C    P2    62W / 180W |   1928MiB /  8114MiB |    100%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [20]:
### Imports ####################################################################
%matplotlib inline
import os
import numpy as np
import pandas as pd
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import matplotlib.pyplot as plt
import theano
import theano.tensor as T
import keras
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Dense, GRU
from typing import Tuple, Callable, List, Optional, Sequence
from tracker import visuals, extractor, utils, metrics

np.set_printoptions(suppress=True)
Tensor = theano.tensor.Tensor

Using cuDNN version 5110 on context None
Mapped name None to device cuda: GeForce GTX 1080 (0000:04:00.0)
Using Theano backend.


In [26]:
def construct():
    for element in 

18


In [25]:
### File Writers ###############################################################
def write4d(file, array: np.ndarray) -> None:
    for cube in array:
        for matrix in cube:
            for row in matrix:
                for number in row:
                    string = "{0: >6.2f} | ".format(number)
                    string = string.replace("-0.00", "     ")
                    string = string.replace( "0.00",  "    ")
                    file.write(string)
                file.write("\n")
            file.write(("-" * 110) + "\n")
        file.write(("=" * 110) + "\n" + ("=" * 110) + "\n")

def write3d(file, array: np.ndarray) -> None:
    for matrix in array:
        for row in matrix:
            for number in row:
                string = "{0: >6.2f} | ".format(number)
                string = string.replace("-0.00", "     ")
                string = string.replace( "0.00",  "    ")
                file.write(string)
            file.write("\n")
            file.write(("-" * 110) + "\n")
        file.write(("=" * 110) + "\n" + ("=" * 110) + "\n")

def write(filename: str, array: np.ndarray) -> None:
    with open(filename, "w") as file:
        if len(array.shape) == 4:
            write4d(file, array)
        elif len(array.shape) == 3:
            write3d(file, array)
        else:
            file.write(str(array))

In [ ]:
### Load in Data ###############################################################
order  = ["phi", "r", "z"]
frame  = pd.read_csv("data/sets/ACTS-MU10-PT1000-T50-PREPARED.gz")
data   = extractor.extract_input(frame, order)
matrix = extractor.extract_output(frame, order)
input_shape  = data.shape[1:]
output_shape = matrix.shape[1:]
print(len(data))
print(input_shape)
print(output_shape)
n = 3
visuals.display_matrices(data[n], matrix[n], order=order, noise=False)

In [ ]:
### Run this cell if you want all events to have max number of hits ############
def regularize_frame(
        frame      : pd.DataFrame,
        num_layers : int,  # How many layers (hits per track).
        num_tracks : int,  # How many tracks per event.
        ) -> pd.DataFrame:
    regular = []
    events = (e for _, e in frame[frame["padding"] == 0].groupby("event_id"))
    for event in events:
        groups = event.groupby("cluster_id")
        tracks = groups.filter(lambda track: num_layers == len(track))
        if num_tracks <= len(pd.unique(tracks["cluster_id"])):
            ids    = tracks["cluster_id"]
            unique = np.sort(pd.unique(ids))
            mapped = dict((v, u) for (u, v) in enumerate(unique))
            tracks = tracks.assign(cluster_id=ids.map(mapped))
            tracks = tracks[tracks["cluster_id"] < num_tracks]
            if num_tracks == len(pd.unique(tracks["cluster_id"])):
                regular.append(tracks)
    return pd.concat(regular)

frame  = regularize_frame(frame, 4, 5)
data   = extractor.extract_input(frame, order)
matrix = extractor.extract_output(frame, order)
input_shape  = data.shape[1:]
output_shape = matrix.shape[1:]
print(input_shape)
print(output_shape)
n = 0
visuals.display_matrices(data[5], matrix[5], order=order, noise=False, padding=False)

In [ ]:
visuals.Plot2D(frame[frame["event_id"] == 0], order).plot(mode="xy", title="")

In [ ]:
#### LossFunctionCreator #######################################################
class LossFunctionCreator:
    def __init__(self,
            input_tensor : Tensor,
            input_shape  : Tuple,
            output_shape : Tuple,
            order        : List[str],
            ) -> None:
        """ Initialize the instance variables. """
        self.__name__     = "LossFunctionCreator"
        self.input_tensor = T.as_tensor_variable(input_tensor)
        self.meshgrid     = self.make_meshgrid(output_shape)
        self.input_shape  = input_shape
        self.output_shape = output_shape
        self.order        = order

    def make_meshgrid(self,
            shape : Tuple[int, int],
            ) -> Tensor:
        """
        Create a meshgrid.
        Example for shape (3, 5):
            [[0, 1, 2, 3, 4],
             [0, 1, 2, 3, 4],
             [0, 1, 2, 3, 4]]
        """
        return (T.mgrid[0:shape[0], 0:shape[1]][1])
    
    def regression_loss(self,
            y_true : Tensor,
            y_pred : Tensor,
            ) -> Tensor:
        input_tensor = self.input_tensor
        tensor = None
        for i in range(output_shape[1]):  # For each track...
            pred_mask = self.get_track_mask(y_pred, i)
            true_mask = self.get_track_mask(y_true, i)
            pred_num_hits = pred_mask.sum(-1).sum(-1) + 2  # Avoid Div(0).
            true_num_hits = true_mask.sum(-1).sum(-1) + 2  # Avoid Div(0).
            pred_masked = pred_mask * input_tensor
            true_masked = true_mask * input_tensor
            pred_line = self.linear_regression(pred_masked, pred_num_hits)
            true_line = self.linear_regression(true_masked, true_num_hits)
            diff   = (pred_line - true_line)**2
            tensor = diff if tensor is None else tensor + diff
        return tensor
    
    def softmax(self,
            tensor     : Tensor,
            axis       : Optional[int] = None,
            refinement : float = 1,
            ) -> Tensor:
        """
        Return the softmax of the tensor along the specified axis.
        Higher refinement yields sharper, more accurate results, but also
        tends to yield NaNs for large tensor values.
        """
        exponent = (refinement * tensor).exp()
        return exponent / exponent.sum(axis=axis, keepdims=True)

    def softargmax(self,
            tensor     : Tensor,
            indices    : Tensor,
            axis       : Optional[int] = None,
            refinement : float = 1,
            ) -> Tensor:
        """
        Return the argsoftmax of the tensor along the specified axis.
        Higher refinement yields sharper, more accurate results, but also
        tends to yield NaNs for large tensor values.
        """
        return (self.softmax(tensor, axis, refinement) * indices).sum(axis)
    
    def get_order_mask(self,
            string : str,
            ) -> Tensor:
        """
        Return a mask such that when output is multiplied by this mask,
        only the column corresponding to the *string* category remains.
        """
        mask = np.zeros(len(self.order))
        mask[self.order.index(string)] = 1
        return T.as_tensor_variable(mask)
    
    def get_track_mask(self,
            output   : Tensor,
            track_id : int,
            aref     : int = 32,  # Refinement value for softargmax.
            mref     : int =  4,  # Refinement value for mask values.
            ) -> Tensor:
        """
        Retrieve a tensor containing a mask such that if self.tensor_input
        was multiplied by the mask, the result would be a tensor containing
        the positions of all hits with the specified track_id.
        """
        cats = self.softargmax(output, self.meshgrid, refinement=aref, axis=-1)
        fill = T.fill(cats, track_id)
        diff = (cats - fill)**2
        mask = 1 / (mref * diff).exp()
        mask = mask.reshape((*T.shape(mask), 1))
        return mask
    
    def linear_regression(self,
            tensor : Tensor,
            length : Tensor,
            ) -> Tensor:
        """
        Given a tensor, and the number of hits within the tensor,
        return the two parameters (m, b) of the least squares
        regression line with equation f[x] = (m * x) + b.
        """
        e = 2 * K.common.epsilon()  # Epsilon to avoid division by 0.
        p = (tensor * self.get_order_mask("phi")).sum(-1)
        r = (tensor * self.get_order_mask("r")).sum(-1)  # r values.
        z = (tensor * self.get_order_mask("z")).sum(-1)  # z values.
        d = (length * (r**2).sum(-1)) - r.sum(-1)**2 + e  # Denominator.
        m = (length * (r * z).sum(-1) - r.sum(-1) * z.sum(-1)) / d
        b = (z.sum(-1) * (r**2).sum(-1) - r.sum(-1) * (r * z).sum(-1)) / d
        return m + b
    
    def __call__(self) -> Callable[[Tensor, Tensor], Tensor]:
        """ Return a Tensor that measures the loss of a model. """
        def custom_loss(y_true: Tensor, y_pred: Tensor) -> Tensor:
            return self.regression_loss(y_true, y_pred)
        return custom_loss

In [ ]:
A = T.dtensor3("A")
B = T.dtensor3("B")
C = T.dtensor3("C")
D = LossFunctionCreator(A, input_shape, output_shape, order)()
E = D(B, C)
F = theano.function([A, B, C], E, on_unused_input='ignore')
# print(theano.printing.debugprint(E))

In [ ]:
positions   = data  [0:4]
true_matrix = matrix[0:4]
pred_matrix = np.random.rand(*true_matrix.shape)
pred_matrix = pred_matrix / pred_matrix.sum(-1, keepdims=True)
evaluation = F(positions, true_matrix, pred_matrix).round(2)
print(evaluation)

In [ ]:
visuals.display_matrices(data[0], true_matrix[0], 2, order, 0, 0)
visuals.display_matrices(data[0], pred_matrix[0], 2, order, 0, 0)

In [ ]:
### Define Model ###############################################################
input_layer = Input(name="Input", shape=input_shape)
model_layer = Dense(name="Dense 1", units=512, activation="relu")(input_layer)
model_layer = Dense(name="Dense 2", units=512, activation="relu")(model_layer)
model_layer = Dense(name="Dense 3", units=512, activation="relu")(model_layer)
output_layer = Dense(name="Softmax", units=output_shape[1],
                     activation="softmax", kernel_initializer="uniform"
                    )(model_layer)
model = Model(inputs=input_layer, outputs=output_layer)
loss  = LossFunctionCreator(input_layer, input_shape, output_shape, order)()
opt   = keras.optimizers.RMSprop(lr=0.0000001)
model.compile(loss=loss, optimizer=opt, metrics=["accuracy"])
model.summary()

In [ ]:
### Fit Model ##################################################################
histories = model.fit(
    data, 
    matrix, 
    epochs=64, 
    batch_size=32,
    verbose=2, 
    validation_data=(data, matrix)
)
predictions = model.predict(data[0:2])
write("output.txt", predictions)

In [ ]:
#### Graph Loss ################################################################
plt.plot(histories.history['loss'])
plt.plot(histories.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train', 'Test'], loc='upper right')
plt.show()

In [ ]:
#### Graph Accuracy ############################################################
plt.plot(histories.history['acc'])
plt.plot(histories.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()